In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("/Users/mareikekeller/air_quality/data/cleaned_data.csv")
df.head()

In [ ]:
# Data preparation: Manipulating the 'Date' column

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Split 'Date' column into 'year', 'month' and 'day'
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Remove 'Date' column
if 'Date' in df.columns:
    df.drop(columns=['Date'], inplace=True)

In [ ]:
# Daten für 2014 & 2025 entfernen, weil zu wenige Datenpunkte

df = df[(df["year"] > 2014) & (df["year"] < 2025)]

In [ ]:
# Tehran komplett entfernen, weil die Schadstoffwerte zu sehr von allen übrigen Städten abweichen
df = df[df["City"] != "Tehran"]

In [ ]:
# Display the first 5 rows of the dataframe
df.head()

In [ ]:
# Spalten mit zu vielen fehlenden Werten entfernen

# Berechnen, wie viele Prozent der Werte pro Spalte fehlen
missing_percent = df.isna().mean() * 100  

# Spalten auswählen, die weniger als 50% fehlende Werte haben
df_cleaned = df.loc[:, missing_percent <= 53]

# Ergebnis ausgeben
print(f"Anzahl der entfernten Spalten: {df.shape[1] - df_cleaned.shape[1]}")
print("Übrige Spalten:", df_cleaned.columns)

In [ ]:
weather_vars = ["dew", "humidity", "pressure", "temperature", "wind-speed"]

df_weather = df.groupby("City")[weather_vars].mean()
df_weather.head()

In [ ]:
# Fehlende Werte interpolieren: lineare Interpolation; die ist möglich, weil wir Daten aus mehreren Jahren haben, d.h. wir schätzen die fehlenden Werte anhand der Werte, die vor und nach dem fehlenden Wert liegen

df_weather.interpolate(method="linear", inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Wetterdaten skalieren (K-Means ist empfindlich gegenüber Skalen)
scaler = StandardScaler()
df_weather_scaled = scaler.fit_transform(df_weather)

# Optimale Clusteranzahl bestimmen (Elbow-Methode)
wcss = []
for k in range(1, 31):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_weather_scaled)
    wcss.append(kmeans.inertia_)

# Elbow-Plot anzeigen
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(range(1, 31), wcss, marker="o", linestyle="--")
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("WCSS (Within-Cluster Sum of Squares)")
plt.title("Elbow-Methode zur Bestimmung der optimalen Clusteranzahl")
plt.show()


In [ ]:
# K-Means mit k=5 durchführen
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_weather["Weather_Cluster"] = kmeans.fit_predict(df_weather_scaled)

In [ ]:
# Neue Cluster-Zuordnung der Städte anzeigen
df_weather_numbers = df_weather[["Weather_Cluster"]].sort_values(by="Weather_Cluster")
df_weather_numbers

In [ ]:
df_cluster = pd.read_csv("df_cluster.csv", index_col=0)  # Index wiederherstellen
df_cluster.head()  # Überprüfen, ob die Daten korrekt geladen wurden


In [ ]:
df_combined = df_weather.merge(df_cluster, left_index=True, right_index=True)



In [ ]:
df_combined.head()

In [ ]:
len(df_combined)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(x="Weather_Cluster", y="pm25", data=df_combined)
plt.xlabel("Wetter-Cluster")
plt.ylabel("PM2.5-Wert")
plt.title("Luftverschmutzung pro Wetter-Cluster")
plt.show()


In [ ]:
# Abweichung vom Cluster-Durchschnitt berechnen
df_combined["pm25_deviation"] = df_combined.groupby("Weather_Cluster")["pm25"].transform(lambda x: x - x.mean())

# Städte mit deutlich höherer PM2.5-Belastung als ihr Cluster-Durchschnitt
outliers = df_combined[df_combined["pm25_deviation"] > df_combined["pm25_deviation"].std() * 1.5]

outliers


In [ ]:
# Sicherstellen, dass "City" eine normale Spalte ist
df_combined = df_combined.reset_index()

# Scatterplot erstellen
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(
    x=df_combined["temperature"], 
    y=df_combined["pm25"], 
    hue=df_combined["Weather_Cluster"], 
    palette="Set1", 
    alpha=0.7
)

# Städtenamen als Labels hinzufügen
for i, row in df_combined.iterrows():
    plt.text(row["temperature"], row["pm25"], row["City"], fontsize=9, ha="right", alpha=0.7)

# Achsen & Titel
plt.xlabel("Temperatur (°C)")
plt.ylabel("PM2.5-Wert")
plt.title("Zusammenhang zwischen Temperatur und Luftverschmutzung")
plt.legend(title="Wetter-Cluster")

plt.show()


Abu Dhabi 🇦🇪	Wüstenstaub (natürliche PM2.5), Öl-Industrie, wenig Wind

Delhi 🇮🇳	Verkehr, Industrie, Kohlekraftwerke, Wetter-Inversionen

Shijiazhuang 🇨🇳	Starke Industrialisierung (Stahlproduktion), Kohlekraftwerke

Ulan Bator 🇲🇳	Kohleheizungen im Winter, extreme Kälte, Luftstagnation

## Analyse zum Einfluss der Jahreszeit

In [ ]:
df_city_avg = df.groupby("City")[["year", "month", "pm25"]].mean().reset_index()
df_city_avg.head()

In [ ]:
len(df_city_avg)

In [ ]:
df_combined.head()

In [ ]:
len(df_combined)

In [ ]:
# Nur die relevanten Spalten behalten
df_monthly = df[["City", "year", "month", "pm25"]].copy()

# Sicherstellen, dass "City" ein String ist
df_monthly["City"] = df_monthly["City"].astype(str)
df_combined["City"] = df_combined["City"].astype(str)

# Merge mit den Wetter-Clustern aus df_combined
df_with_month = df_monthly.merge(df_combined[["City", "Weather_Cluster", "Cluster"]], on="City", how="left")

# Überprüfen, ob der Merge funktioniert hat
print(df_with_month.head())
print(df_with_month.isna().sum())  # Prüfen, ob Werte fehlen



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_with_month, x="month", y="pm25", hue="City", marker="o")
plt.xlabel("Monat")
plt.ylabel("PM2.5-Wert")
plt.title("Luftverschmutzung (PM2.5) in verschiedenen Monaten")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.legend(title="Stadt")
plt.show()


In [ ]:
# Nur die drei relevanten Städte auswählen
selected_cities = ["Delhi", "Shijiazhuang", "Ulan Bator", "Abu Dhabi"]
df_winter = df_with_month[df_with_month["City"].isin(selected_cities)]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_winter, x="month", y="pm25", hue="City", marker="o", palette="Set1")
plt.xlabel("Monat")
plt.ylabel("PM2.5-Wert")
plt.title("Luftverschmutzung (PM2.5) über das Jahr – Vergleich für Delhi, Shijiazhuang & Ulan Bator")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.legend(title="Stadt")

# plt.savefig("../Images/pm25_per_month_top_4.png", dpi=300, bbox_inches="tight")

plt.show()
